# Classification automatique de documents par les techniques de Machine Learning 


Les travaux présentés dans ce notebook traitent de la classification automatique de documents, et particulièrement de la classification supervisée puisqu’elle opère à partir d’un ensemble de classes prédéfinies. L’approche proposée repose sur une représentation vectorielle des documents axée sur les mots contenus, mais aussi sur une représentation plus sémantique de ceux-ci.

## Création du jeu de données

* On essai d'avoir un jeu de données tiré du site 'https://www.groupedeveillecovid.fr/cat/articles-scientifiques/'
* Le but étant de classer chaque nouveau document pdf dans la catégorie correspondante et ainsi faire une prediction correcte.


### Web Scraping

* Il faut savoir que le web scraping est une technique permettant l’extraction des données d’un site via un programme ou un logiciel. Dans notre cas, nous allons passé vers une bibliothèque Python d'analyse syntaxique de documents HTML et XML. L’objectif est donc d’extraire le contenu d’une page d’un site de façon structurée. Le scraping permet ainsi de pouvoir réutiliser ces données.
* Pour la suite de notre processus, nous avons utlisé plusieurs librairie. <br/> Nous allons en détaillé quelques un. Nous avons par exemple : <br/> 
    * request : <br/> 
          La bibliothèque de 'request' est la norme de facto pour effectuer des requêtes HTTP en Python. Elle résume la complexité des requêtes derrière une API simple et élégante afin de pouvoir se concentrer sur l'interaction avec les services et l'utilisation de données dans votre application.

    * PyPDF : <br/> 
           Un package pour extraire du texte et des métadonnées d'un fichier PDF.
    * Beautiful Soup : <br/> 
            Bibliothèque Python d'analyse syntaxique de documents HTML et XML.
        

# Importation des données

In [1]:
import requests,re 
import numpy as np
import pandas as pd
import io

import os
import math
import time

import matplotlib.pyplot as plt
import pickle
import seaborn as sns
sns.set_style("whitegrid")
import altair as alt
import sklearn.metrics as metrics
import bs4

import plotly.figure_factory as ff
import plotly.graph_objects as go
import plotly.express as px

# Below libraries are for feature representation using sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder 


# Multinomial Naive Bayes

from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split

from sklearn.neural_network import MLPClassifier

from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from sklearn.svm import SVC  

from sklearn.metrics import classification_report, confusion_matrix  
from sklearn import svm
from sklearn.model_selection import GridSearchCV



from bs4 import BeautifulSoup
from PyPDF2 import PdfFileReader


# Below libraries are for text processing using NLTK
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Below libraries are for similarity matrices using sklearn
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

# Initialisation des variables

In [2]:
# Init variables
global df, merged, dts
urls_to_scrape = []
urls_to_scrape_1 = []
liste_des_articles = []
category_articles = []
liste_url_pdf = []
list_content = []

date = ''
auteur = ''
auteurs = ''
lien = ''

dataset_list = []
# find ".pdf" in a string
match = re.compile('\.(pdf)')

datasetObj = pd.DataFrame()


# Fonction pour extraire les informations qui vont nous servir dans un fichier pdf

In [3]:
def extract_df(url, category, title,auteur,lien,date):
    global df_information
    r = requests.get(url)
    f = io.BytesIO(r.content)

    reader = PdfFileReader(f)
    #contents = reader.getPage(0).extractText().split('\n')

    information = reader.getDocumentInfo()
    number_of_pages = reader.getNumPages()
    content = ''
    for x in range(0, number_of_pages):
        contents=''.join(reader.getPage(x).extractText().split('\n'))
        #content = reader.getPage(x).extractText().split('\n')
        content = contents
        df_information = pd.DataFrame(
            {'Category': [category], 'Title': [str(title)],'Auteur': [str(auteur)], 'Lien': [str(lien)], 'Number of pages': [number_of_pages], 'Content': [str(content)],'Date': [str(date)]})
        # print(type(df_information))
    return df_information

#### Fonction pour parcourir tout le site et extraire tout les fichier pdf 
#### Ces fichiers sont rentré dans la fonction précédentes pour sortir les informations suivantes dans un dataframe :
     - La catégorie 
     - Le titre du fichier pdf 
     - Le nombre de page du fichier 
     - L'auteur
     - le contenu en texte du fichier

In [4]:
def extract_from_url():
    # Request Page
    request_url = "https://www.groupedeveillecovid.fr/cat/articles-scientifiques/"
    requested_page = requests.get(request_url)
    page_content = requested_page.text
    
    # Instantiate BeautifulSoup
    soup_object = BeautifulSoup(page_content, features="lxml")
    tag_cloud_elements = soup_object.find_all("div", {"class": "tagcloud"})

    del urls_to_scrape[:]
    del urls_to_scrape_1[:]
    del liste_url_pdf[:]
    # Iterate through the elements
    for div_tag in tag_cloud_elements:
        for tag in div_tag.find_all('a', href=True):
            urls_to_scrape.append(tag['href'])
    
    # Scrape all titles and abstracts
    for category_articles_url in range(len(urls_to_scrape)):
        
        # Build the category name
        category_name = urls_to_scrape[category_articles_url][:-1]  # Removes the last trailing slash
        category_name = category_name.rsplit('/', 1)[1]  # Get the last part of the URL after a "/"
        
        requested_page_1 = requests.get(urls_to_scrape[category_articles_url])
        page_content_1 = requested_page_1.text
        
        
           
        
        # Instantiate BeautifulSoup
        soup_object_1 = BeautifulSoup(page_content_1, features="lxml")
        tag_cloud_elements_1 = soup_object_1.find_all("div", {"class": "btReadArticle"})
        category_abstract_elements = soup_object_1.find_all("span", {"class": "headline"})

        #print(urls_to_scrape[category_articles_url])
        # Iterate through the elements
        
        for div_tag_1 in range(len(tag_cloud_elements_1)):
            #print(category_abstract_elements[div_tag_1].text)
            for tag_1 in tag_cloud_elements_1[div_tag_1].find_all('a', href=True):
                #print(tag_1['href'])
                # Instantiate BeautifulSoup
                  
                requested_page_2 = requests.get(tag_1['href'])
                page_content_2 = requested_page_2.text

                # Instantiate BeautifulSoup
                soup_object_2 = BeautifulSoup(page_content_2, features="lxml")
                tag_cloud_elements_titre = soup_object_2.find_all("span", {"class": "btArticleDate"})
                tag_cloud_elements_auteur = soup_object_2.find(class_= "btText").find_all('p')
                tag_cloud_elements_auteurs = soup_object_2.find(class_= "btText").find_all('em')

                lien = tag_1['href']
                date = tag_cloud_elements_titre[0].text
                #auteurs = tag_cloud_elements_auteurs.text
                
                #print(tag_cloud_elements_auteur)
                
                if not tag_cloud_elements_auteurs:
                    try:
                        for x in tag_cloud_elements_auteur[1]:

                            if type(x) is not  bs4.element.Tag:
                                try:
                                    cont = x.split(':')[1]
                                except IndexError:
                                    cont = 'null'
                                auteur = cont.lstrip().rstrip('.')
                            else:
                                auteur = 'null'
                    except IndexError:
                        auteur = 'null'
                else:
                    for x in tag_cloud_elements_auteurs:
                        auteur = x.text.lstrip().rstrip('.')
                    
                 
                
                #print(lien)
                #print(date)      
                #print(auteur)
                #print('---------------------------')
              
                # check links
                for link in soup_object_2.findAll('a'):
                    try:
                        href = link['href']
                        if re.search(match, href):
                            # liste_url_pdf.append(href)
                            # print(href)
                            # Lire le contenu du fichier pdf
                            df = extract_df(href,category_name,category_abstract_elements[div_tag_1].text,auteur,lien,date)
                            #df = extract_df(href, category_name,category_abstract_elements[div_tag_1].text)
                            list_content.append(df)
                            #print('--------- PRINTING THE FINAL DATASET --------------------')
                            merged = pd.concat(list_content)
                            #print('-------------------------------------')
                            #print(type(merged))
                            #print(merged)
                    except KeyError:
                        pass
            # urls_to_scrape_1.append(tag_1['href'])
            #print(title_name)
                

    return merged
#if __name__ == '__main__':
 #   extract_from_url()
    
    
    
    

#### On récuperer le dataset grace au deux fontions précédentes

In [5]:
dts = extract_from_url()
dts.head()


,Category,Title,Auteur,Lien,Number of pages,Content,Date
0,biologie,"Current Status of Epidemiology, Diagnosis, The...",Dae-Gyun Ahn and al,https://www.groupedeveillecovid.fr/blog/2020/0...,12,"324Ahn et al.J. Microbiol. Biotechnol.'˛˛ ˜,˜ ...",21 avril 2020
0,biologie,Into the Eye of the Cytokine Storm,Jennifer R. Tisoncik et al,https://www.groupedeveillecovid.fr/blog/2020/0...,17,"ThomasR.Martin,M.D.,isEmeritusProfessorofMedic...",10 avril 2020
0,biologie,Lymphopenic community acquired pneumonia as si...,Bermejo-Martin and al,https://www.groupedeveillecovid.fr/blog/2020/0...,2,2 Letter to the Editor / Journal of Infection ...,10 avril 2020
0,biologie,Functional exhaustion of antiviral lymphocytes...,Zheng et al,https://www.groupedeveillecovid.fr/blog/2020/0...,3,ADDITIONALINFORMATIONTheonlineversionofthisart...,10 avril 2020
0,biologie,Hematologic parameters in patients with COVID-...,Fan et al,https://www.groupedeveillecovid.fr/blog/2020/0...,8,<100 100-150 >150 0 (0.0) 15 (25.9) 43 (74.1...,10 avril 2020


In [6]:
dts.to_csv("train_data_recommendation.csv", sep=';', encoding='utf-8')

In [7]:
data=pd.read_csv('train_data_recommendation.csv',sep=";")
del data['Unnamed: 0']
data.head(6)

,Category,Title,Auteur,Lien,Number of pages,Content,Date
0,biologie,"Current Status of Epidemiology, Diagnosis, The...",Dae-Gyun Ahn and al,https://www.groupedeveillecovid.fr/blog/2020/0...,12,"324Ahn et al.J. Microbiol. Biotechnol.'˛˛ ˜,˜ ...",21 avril 2020
1,biologie,Into the Eye of the Cytokine Storm,Jennifer R. Tisoncik et al,https://www.groupedeveillecovid.fr/blog/2020/0...,17,"ThomasR.Martin,M.D.,isEmeritusProfessorofMedic...",10 avril 2020
2,biologie,Lymphopenic community acquired pneumonia as si...,Bermejo-Martin and al,https://www.groupedeveillecovid.fr/blog/2020/0...,2,2 Letter to the Editor / Journal of Infection ...,10 avril 2020
3,biologie,Functional exhaustion of antiviral lymphocytes...,Zheng et al,https://www.groupedeveillecovid.fr/blog/2020/0...,3,ADDITIONALINFORMATIONTheonlineversionofthisart...,10 avril 2020
4,biologie,Hematologic parameters in patients with COVID-...,Fan et al,https://www.groupedeveillecovid.fr/blog/2020/0...,8,<100 100-150 >150 0 (0.0) 15 (25.9) 43 (74.1...,10 avril 2020
5,biologie,Characteristics of peripheral lymphocyte subse...,Fan Wang et al,https://www.groupedeveillecovid.fr/blog/2020/0...,20,"Accepted Manuscript 19. Cannon MJ, Stott EJ,...",10 avril 2020


In [8]:
data['Date']

0      21 avril 2020 
1      10 avril 2020 
2      10 avril 2020 
3      10 avril 2020 
4      10 avril 2020 
            ...      
134      23 mai 2020 
135      23 mai 2020 
136       3 mai 2020 
137       3 mai 2020 
138    25 avril 2020 
Name: Date, Length: 139, dtype: object

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
Category           139 non-null object
Title              139 non-null object
Auteur             127 non-null object
Lien               139 non-null object
Number of pages    139 non-null int64
Content            134 non-null object
Date               139 non-null object
dtypes: int64(1), object(6)
memory usage: 7.7+ KB


In [10]:
# convert the 'Date' column to datetime format 
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import parsedatetime as pdt
import dateparser

#data['Date'] = data['Date'].astype('str') 


In [11]:
calendar = pdt.Calendar(pdt.Constants(localeID='fr', usePyICU=True))
for date_string in data['Date']:
    dt, success = calendar.parseDT(date_string)
    if success:
        data['DateConvertit']= dt.date()
       #print(date_string, dt.date())

In [12]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 8 columns):
Category           139 non-null object
Title              139 non-null object
Auteur             127 non-null object
Lien               139 non-null object
Number of pages    139 non-null int64
Content            134 non-null object
Date               139 non-null object
DateConvertit      139 non-null object
dtypes: int64(1), object(7)
memory usage: 8.8+ KB


In [13]:
data['Date'].head()
data['Date'] = data['Date'].astype(str)
del data['DateConvertit']

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
Category           139 non-null object
Title              139 non-null object
Auteur             127 non-null object
Lien               139 non-null object
Number of pages    139 non-null int64
Content            134 non-null object
Date               139 non-null object
dtypes: int64(1), object(6)
memory usage: 7.7+ KB


In [15]:
import dateparser # $ pip install dateparser
data['Date'] = data['Date'].apply(dateparser.parse)

#for date_string in data['Date']:
#    print(dateparser.parse(date_string).date())


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139 entries, 0 to 138
Data columns (total 7 columns):
Category           139 non-null object
Title              139 non-null object
Auteur             127 non-null object
Lien               139 non-null object
Number of pages    139 non-null int64
Content            134 non-null object
Date               139 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 7.7+ KB


In [17]:
data

,Category,Title,Auteur,Lien,Number of pages,Content,Date
0,biologie,"Current Status of Epidemiology, Diagnosis, The...",Dae-Gyun Ahn and al,https://www.groupedeveillecovid.fr/blog/2020/0...,12,"324Ahn et al.J. Microbiol. Biotechnol.'˛˛ ˜,˜ ...",2020-04-21
1,biologie,Into the Eye of the Cytokine Storm,Jennifer R. Tisoncik et al,https://www.groupedeveillecovid.fr/blog/2020/0...,17,"ThomasR.Martin,M.D.,isEmeritusProfessorofMedic...",2020-04-10
2,biologie,Lymphopenic community acquired pneumonia as si...,Bermejo-Martin and al,https://www.groupedeveillecovid.fr/blog/2020/0...,2,2 Letter to the Editor / Journal of Infection ...,2020-04-10
3,biologie,Functional exhaustion of antiviral lymphocytes...,Zheng et al,https://www.groupedeveillecovid.fr/blog/2020/0...,3,ADDITIONALINFORMATIONTheonlineversionofthisart...,2020-04-10
4,biologie,Hematologic parameters in patients with COVID-...,Fan et al,https://www.groupedeveillecovid.fr/blog/2020/0...,8,<100 100-150 >150 0 (0.0) 15 (25.9) 43 (74.1...,2020-04-10
...,...,...,...,...,...,...,...
134,epidemiologie-et-sante-publique,Individual quarantine versus active monitoring...,“COVID-19 : when should quarantine be enforced“,https://www.groupedeveillecovid.fr/blog/2020/0...,2,2 www.thelancet.com/infection Published onli...,2020-05-23
135,epidemiologie-et-sante-publique,Individual quarantine versus active monitoring...,“COVID-19 : when should quarantine be enforced“,https://www.groupedeveillecovid.fr/blog/2020/0...,9,www.thelancet.com/infection Published online...,2020-05-23
136,epidemiologie-et-sante-publique,Epidemiological characteristics and incubation...,X Nie et al,https://www.groupedeveillecovid.fr/blog/2020/0...,30,Accepted Manuscript 30 Figure 4 Downloaded f...,2020-05-03
137,epidemiologie-et-sante-publique,Early dynamics of transmission and control of ...,Adam J Kucharski et al,https://www.groupedeveillecovid.fr/blog/2020/0...,6,558 www.thelancet.com/infection Vol 20 May...,2020-05-03


# Vérification et suppression de tous les doublons

In [18]:
data.sort_values('Title',inplace=True, ascending=False)

duplicated_articles_series = data.duplicated('Title', keep = False)

data = data[~duplicated_articles_series]
print("Nombre total d'articles après élimination des doublons:", data.shape[0])

Nombre total d'articles après élimination des doublons: 113


# Vérification des valeurs manquantes

In [19]:
data.isna().sum()

Category            0
Title               0
Auteur             12
Lien                0
Number of pages     0
Content             4
Date                0
dtype: int64

In [20]:
data_propre = data.dropna()
data_propre.isna().sum()

Category           0
Title              0
Auteur             0
Lien               0
Number of pages    0
Content            0
Date               0
dtype: int64

In [21]:
data_propre.shape

(98, 7)

# Exploration des données de base

## Statistiques de base - Nombre d'articles, d'auteurs, de catégories

In [22]:
print("Nombre total d'articles : ", data_propre.shape[0])
print("Nombre total d'auteurs : ", data_propre["Auteur"].nunique())
print("Nombre total de catégories unqiue : ", data_propre["Category"].nunique())

Nombre total d'articles :  98
Nombre total d'auteurs :  93
Nombre total de catégories unqiue :  28


## Répartition des articles par catégorie

In [24]:
fig = go.Figure([go.Bar(x=data_propre["Category"].value_counts().index, y=data_propre["Category"].value_counts().values)])

fig['layout'].update(title={"text" : 'Répartition des articles par catégorie','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Nom de la categorie",yaxis_title="Nombre d'articles")

fig.update_layout(width=800,height=700)

fig

## Nombre d'articles par mois

In [25]:
data_propre_mois = data_propre.resample('m',on = 'Date')['Title'].count()
data_propre_mois

Date
2020-03-31    17
2020-04-30    50
2020-05-31    25
2020-06-30     6
Freq: M, Name: Title, dtype: int64

In [26]:
fig = go.Figure([go.Bar(x=data_propre_mois.index.strftime("%b"), y=data_propre_mois)])
fig['layout'].update(title={"text" : 'Répartition des articles par mois','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Moid",yaxis_title="Nombre d'articles")
fig.update_layout(width=500,height=500)
fig

## PDF for the length of headlines

In [27]:
fig = ff.create_distplot([data_propre['Title'].str.len()], ["ht"],show_hist=False,show_rug=False)
fig['layout'].update(title={'text':'PDF','y':0.9,'x':0.5,'xanchor': 'center','yanchor': 'top'}, xaxis_title="Longueur d'un titre",yaxis_title="probabilité")
fig.update_layout(showlegend = False,width=500,height=500)
fig
## distribution Gaussienne

In [28]:
data_propre.index = range(data_propre.shape[0])

In [29]:
# Ajouter une nouvelle colonne contenant à la fois le jour de la semaine et le mois, 
#elle sera nécessaire ultérieurement tout en recommandant sur la base du jour de la 
#semaine et du mois

data_propre["jour et mois"] = data_propre["Date"].dt.strftime("%a") + "_" + data_propre["Date"].dt.strftime("%b")


In [30]:
data_propre_temp = data_propre.copy()
data_propre_temp


,Category,Title,Auteur,Lien,Number of pages,Content,Date,jour et mois
0,bloc-operatoire,What we do when a COVID-19 patient needs an op...,Lian Kah Ti and Al,https://www.groupedeveillecovid.fr/blog/2020/0...,3,2.GovernmentofSingapore.Coronavirusdisease2019...,2020-03-26,Thu_Mar
1,telemedecine,Virtually Perfect? Telemedicine for Covid-19,"Judd E. Hollander, and Brendan G. Carr",https://www.groupedeveillecovid.fr/blog/2020/0...,3,PERSPECTIVE3Virtually Perfect? Telemedicine fo...,2020-03-24,Tue_Mar
2,virologie,Virological assessment of hospitalized patient...,Roman Wölfel and al,https://www.groupedeveillecovid.fr/blog/2020/0...,17,3nature research | reporting summaryOctober ...,2020-04-08,Wed_Apr
3,virologie-reponse-immunitaire,Viral dynamics in mild and severe cases of COV...,Liu Y et al,https://www.groupedeveillecovid.fr/blog/2020/0...,2,2 www.thelancet.com/infection Published onli...,2020-03-25,Wed_Mar
4,biologie,Updated approaches against SARS -CoV – 2,Haiou Li and al,https://www.groupedeveillecovid.fr/blog/2020/0...,25,"on April 5, 2020 at Universite de Strasbourgh...",2020-04-06,Mon_Apr
...,...,...,...,...,...,...,...,...
93,clinique,An outbreak of severe Kawasaki-like disease at...,Lucio Verdoni and al,https://www.groupedeveillecovid.fr/blog/2020/0...,8,"8 www.thelancet.com Published online May 13,...",2020-05-14,Thu_May
94,covid-et-grossesse,An Analysis of 38 Pregnant Women with COVID-19...,David A. Schwartz,https://www.groupedeveillecovid.fr/blog/2020/0...,25,Table 4. Characteristics of an additional 17 ...,2020-03-27,Fri_Mar
95,covid-et-pathologies-chroniques,Acute hypertriglyceridemia in patients with CO...,Austin R. Morrison and al,https://www.groupedeveillecovid.fr/blog/2020/0...,5,This article is protected by copyright. All r...,2020-04-28,Tue_Apr
96,covid-et-pathologies-chroniques,Acute coronary syndromes during COVID-19,Serafina Valente and al,https://www.groupedeveillecovid.fr/blog/2020/0...,3,PatientsawaitingtheCOVID-19testresultmustbecon...,2020-05-26,Tue_May


# Text Preprocessing

## Suppression des mots d'arrêt
Les mots arrêt ne sont pas très utiles pour l'analyse et leur inclusion prend beaucoup de temps pendant le traitement, alors supprimons-les.

In [31]:
#Les mots arrêt ne sont pas très utiles pour l'analyse et leur inclusion prend 
#beaucoup de temps pendant le traitement, alors supprimons-les.
stop_words = set(stopwords.words('english'))

In [32]:
for i in range(len(data_propre_temp["Title"])):
    string = ""
    for word in data_propre_temp["Title"][i].split():
        word = ("".join(e for e in word if e.isalnum()))
        word = word.lower()
        if not word in stop_words:
          string += word + " "  
    if(i%1000==0):
        print(i)
           # To track number of records processed
    data_propre_temp.at[i,"Title"] = string.strip()
    
    

0


## Lemmatization
La lemmatisation est l'une des techniques de prétraitement de texte les plus courantes utilisées dans le traitement du langage naturel (NLP) et l'apprentissage automatique en général. Cette technique permetréduire un mot donné à sa racine. Le mot racine est appelé une tige dans le processus de stemming.

In [33]:
lemmatizer = WordNetLemmatizer()


In [34]:
for i in range(len(data_propre_temp["Title"])):
    string = ""
    for w in word_tokenize(data_propre_temp["Title"][i]):
        string += lemmatizer.lemmatize(w,pos = "v") + " "
    data_propre_temp.at[i, "Title"] = string.strip()
    if(i%1000==0):
        print(i) 

0


## Utilisation de la méthode du "sac de mots" (Bag of Words)

La manière la plus simple de représenter un document, est de la représenter par un ensemble des mots qu'il contient. En pratique, ça peut être par exemple un vecteur de fréquence d'apparition des différents mots utilisés.
Dans la représentation bag-of-words, on représente chaque document par un vecteur de la taille du vocabulaire |V| et on utilisera la matrice composée de l’ensemble de ces N documents qui forment le corpus comme entrée de nos algorithmes.

In [35]:
headline_vectorizer = CountVectorizer()
headline_features   = headline_vectorizer.fit_transform(data_propre_temp['Title'])

In [36]:
headline_features.get_shape()


(98, 465)

In [37]:
pd.set_option('display.max_colwidth', -1)  # Pour afficher complètement un très long titre

In [38]:
def bag_of_words_based_model(row_index, num_similar_items):
    
    couple_dist = pairwise_distances(headline_features,headline_features[row_index])
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df = pd.DataFrame({'Date de publication': data_propre['Date'][indices].values,
               'Titre':data_propre['Title'][indices].values,
                'Lien':data_propre['Lien'][indices].values,
                "Similitude euclidienne avec l'article interrogé": couple_dist[indices].ravel()})
    print("="*30,"Détails de l'article recherché","="*30)
    print('Titre : ',data_propre['Title'][indices[0]])
    print("\n","="*25,"Articles recommandés : ","="*23)
    #return df.iloc[1:,1]
    return df.iloc[1:,]

bag_of_words_based_model(2, 6) # Modifier l'index des lignes pour tout autre article interrogé

============================== Détails de l'article recherché ==============================
Titre :  Virological assessment of hospitalized patients with COVID-2019

 ========================= Articles recommandés :  =======================


,Date de publication,Titre,Lien,Similitude euclidienne avec l'article interrogé
1,2020-05-11,Observational Study of Hydroxychloroquine in Hospitalized Patients with Covid-19,https://www.groupedeveillecovid.fr/blog/2020/05/11/observational-study-of-hydroxychloroquine-in-hospitalized-patients-with-covid-19/,2.645751
2,2020-04-27,Covid-19 and Kidney Transplantation,https://www.groupedeveillecovid.fr/blog/2020/04/27/covid-19-and-kidney-transplantation/,2.828427
3,2020-05-26,Early Short Course Corticosteroids in Hospitalized Patients with COVID-19,https://www.groupedeveillecovid.fr/blog/2020/05/26/early-short-course-corticosteroids-in-hospitalized-patients-with-covid-19/,2.828427
4,2020-04-10,Into the Eye of the Cytokine Storm,https://www.groupedeveillecovid.fr/blog/2020/04/10/into-the-eye-of-the-cytokine-storm/,2.828427
5,2020-04-10,Surgery in COVID-19 patients: operational directives,https://www.groupedeveillecovid.fr/blog/2020/04/10/surgery-in-covid-19-patients-operational-directives/,2.828427


La fonction ci-dessus recommande 6 articles similaires à l'article recherché en fonction du titre. Elle accepte deux arguments - l'index des articles déjà lus et le nombre total d'articles à recommander.

Sur la base de la distance euclidienne, elle trouve 6 voisins les plus proches et les recommande.

#### Inconvénients de cette méthode

1. Elle accorde une très faible importance aux mots moins fréquemment observés dans le corpus. Peu de mots de l'article interrogé comme "assesment", "patients","hospitalized" apparaissent moins fréquemment dans l'ensemble du corpus, de sorte que la méthode BoW ne recommande aucun article dont le titre contient ces mots. Puisque "COVID" est un mot couramment observé dans le corpus, la méthode BoW recommande les articles dont le titre contient "COVID".
2. La méthode BoW ne préserve pas l'ordre des mots. 

Pour surmonter le premier inconvénient, nous utilisons la méthode TF-IDF pour la représentation des caractéristiques.

## Utilisation de la méthode TF-IDF

La méthode TF-IDF est une mesure pondérée qui donne plus d'importance aux mots moins fréquents dans un corpus. Elle attribue un poids à chaque terme (mot) dans un document en fonction de la fréquence des termes (TF) et de la fréquence inverse des documents (IDF).

TF(i,j) = (# fois que le mot i apparaît dans le document j) / (# mots dans le document j)

IDF(i,D) = log_e(#documents dans le corpus D) / (#documents contenant le mot i)

poids(i,j) = TF(i,j) x IDF(i,D)

Ainsi, si un mot apparaît plus de fois dans un document mais moins de fois dans tous les autres documents, sa valeur TF-IDF sera élevée.

In [39]:
tfidf_headline_vectorizer = TfidfVectorizer(min_df = 0)
tfidf_headline_features = tfidf_headline_vectorizer.fit_transform(data_propre_temp['Title'])

In [40]:
def tfidf_based_model(row_index, num_similar_items):
    couple_dist = pairwise_distances(tfidf_headline_features,tfidf_headline_features[row_index])
    indices = np.argsort(couple_dist.ravel())[0:num_similar_items]
    df = pd.DataFrame({'Date de publication': data_propre['Date'][indices].values,
               'Titre':data_propre['Title'][indices].values,
                'Lien':data_propre['Lien'][indices].values,
                "Similitude euclidienne avec l'article interrogé": couple_dist[indices].ravel()})
    print("="*30,"Détails de l'article recherché","="*30)
    print('Titre : ',data_propre['Title'][indices[0]])
    print("\n","="*25,"Articles recommandés : ","="*23)
    
    #return df.iloc[1:,1]
    return df.iloc[1:,]
tfidf_based_model(2, 6)

============================== Détails de l'article recherché ==============================
Titre :  Virological assessment of hospitalized patients with COVID-2019

 ========================= Articles recommandés :  =======================


,Date de publication,Titre,Lien,Similitude euclidienne avec l'article interrogé
1,2020-05-11,Observational Study of Hydroxychloroquine in Hospitalized Patients with Covid-19,https://www.groupedeveillecovid.fr/blog/2020/05/11/observational-study-of-hydroxychloroquine-in-hospitalized-patients-with-covid-19/,1.235995
2,2020-05-26,Early Short Course Corticosteroids in Hospitalized Patients with COVID-19,https://www.groupedeveillecovid.fr/blog/2020/05/26/early-short-course-corticosteroids-in-hospitalized-patients-with-covid-19/,1.268428
3,2020-04-23,Glycemic Characteristics and Clinical Outcomes of COVID-19 Patients Hospitalized in the United States,https://www.groupedeveillecovid.fr/blog/2020/04/23/6031/,1.277125
4,2020-04-27,"Systematic review of the efficacy and safety of antiretroviral drugs against SARS, MERS or COVID-19: initial assessment",https://www.groupedeveillecovid.fr/blog/2020/04/27/systematic-review-of-the-efficacy-and-safety-of-antiretroviral-drugs-against-sars-mers-or-covid-19-initial-assessment-2/,1.304108
5,2020-05-14,Development and Validation of a Clinical Risk Score to Predict the Occurrence of Critical Illness in Hospitalized Patients With COVID-19,https://www.groupedeveillecovid.fr/blog/2020/05/14/development-and-validation-of-a-clinical-risk-score-to-predict-the-occurrence-of-critical-illness-in-hospitalized-patients-with-covid-19/,1.310307


Par rapport à la méthode BoW, la méthode TF-IDF recommande ici les articles dont le titre contient des mots comme "assesment", "patients","hospitalized" dans les 5 recommandations principales et ces mots apparaissent moins fréquemment dans le corpus.

#### Inconvénients :-

Les méthodes Bow et TF-IDF ne permettent pas de saisir la similarité sémantique et syntaxique d'un mot donné avec d'autres mots, mais celle-ci peut être saisie à l'aide d'encastrements de mots.

Par exemple : il existe une bonne association entre des mots comme "patients" et "hospitalized", "bureau et employé", "tigre" et "léopard", etc. Ce type de similarité sémantique peut être saisi à l'aide de techniques d'intégration de mots. Les techniques d'intégration de mots comme Word2Vec, GloVe et fastText permettent de saisir la similarité sémantique entre les mots.



## Séparation du jeu de données¶
L’entraînement d’un modèle revient à mesurer l’erreur de la sortie de l’algorithme avec les données d’exemple et chercher à la minimiser.
Pour minimiser ce problème,nous avons dès le départ séparé notre jeu de données en deux parties distinctes :

* Le training set, qui va nous permettre d’entraîner notre modèle et sera utilisé par l’algorithme d’apprentissage. C'est celui dont on a parlé depuis le début.
* Le testing set, qui permet de mesurer l’erreur du modèle final sur des données qu’il n’a jamais vues. On va simplement passer ces données comme s'il s'agissait de données que l’on n'a encore jamais rencontrées (comme cela va se passer ensuite en pratique pour prédire de nouvelles données) et mesurer la performance de notre modèle sur ces données. <br/> En effet, le modèle est complètement optimisé pour les données à l'aide desquelles il a été créé. L'erreur sera précisément minimum sur ces données. Alors que l'erreur sera toujours plus élevée sur des données que le modèle n'aura jamais vues. C'est la raison pour laquelle il ne faut pas évaluer la qualité de notre modèle final à l'aide des mêmes données qui ont servi pour l'entraînement. 

Nous avons séparé les données avec les proportions suivantes : 80 % pour le training set et 20 % pour le testing set. 

Nous avons utilisé la fonction de scikit-learn train_test_split qui prend en paramètre la proportion désirée.

## Modélisation des données¶
Pour diffuser des prédictions à partir d'une plateforme, il faut exporter le modèle de machine learning entraîné sous la forme d'un ou plusieurs artefacts. Il existe différentes manières d'exporter des modèles entraînés pour un déploiement sur une plateforme.

Les méthodes d'exportation de modèle s'appliquent quel que soit l'environnement d'entraînement.

Le module Pickle est un moyen d'expoter les modèles de Machine Learning. Il s'agit d'un moyen standard de sérialiser des objets en Python. On utilise l'opération de décapage pour sérialiser vos algorithmes d'apprentissage automatique et enregistrer le format sérialisé dans un fichier.

Plus tard,nous chargerons ces fichiers pour désérialiser notre modèle et l'utiliser pour faire de nouvelles prédictions.

In [41]:
#GET VECTOR COUNT
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform((data_propre.Content).values.astype('U'))
#SAVE WORD VECTOR
pickle.dump(count_vect.vocabulary_, open("count_vector.pkl","wb"))

In [42]:
#TRANSFORM WORD VECTOR TO TF IDF
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

#SAVE TF-IDF
pickle.dump(tfidf_transformer, open("tfidf.pkl","wb"))

## Choix du modèle


Une fois réduit l’espace vectoriel nous procédons au choix du modèle de classification. Ce modèle sera ensuite utilisé pour l’évaluation des textes du jeu de test. Nous avons utilisé plusieurs méthodes de classification. Elles sont fondées sur quatre méthodes principales. Nous avons également tenté d’autres procédures de classification dont les performances se sont révélées moins intéressantes. Le choix de la procédure de classification s’est fait sur chaque ensemble d’apprentissage ou corpus. La sélection fut très simple, nous avons conservé la méthode de classification la plus performante pour un corpus donné.

* La classification probabiliste utilisant la combinaison de la loi de Bayes et de la loi multinomiale
* La classification par les machines à vecteurs support S.V.M.
* La classification par la méthode des réseaux de neurones (Radial Basis Function)



## Multinomial Naive Bayes
Cette technique est classique pour la catégorisation de textes Elle combine l’utilisation de la loi de Bayes bien connue en probabilités et la loi multinomiale. Nous avons simplement précisé le calcul de la loi à priori en utilisant l’estimateur de Laplace pour éviter les biais dus à l’absence de certains mots dans un texte.


In [43]:
# Multinomial Naive Bayes

#clf = MultinomialNB().fit(X_train_tfidf, training_data.flag)
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, data_propre.Category, test_size=0.25, random_state=42)
clf = MultinomialNB().fit(X_train, y_train)

#SAVE MODEL
pickle.dump(clf, open("nb_model.pkl", "wb"))

In [44]:
#LOAD MODEL
loaded_vec = CountVectorizer(vocabulary=pickle.load(open("count_vector.pkl", "rb")))
loaded_tfidf = pickle.load(open("tfidf.pkl","rb"))
loaded_model = pickle.load(open("nb_model.pkl","rb"))

In [45]:
predicted_bayes = loaded_model.predict(X_test)
result_bayes = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted_bayes})
result_bayes.to_csv('res_bayes.csv', sep = ',')

result_bayes.head()

,true_labels,predicted_labels
62,imagerie,epidemiologie-et-sante-publique
40,clinique,covid-et-pediatrie
94,covid-et-grossesse,epidemiologie-et-sante-publique
18,biologie,epidemiologie-et-sante-publique
81,biologie,epidemiologie-et-sante-publique


## Réseau de neurones

In [46]:

clf_neural = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, data_propre.Category, test_size=0.25, random_state=42)

clf_neural.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(15,), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [47]:
metrics.accuracy_score(y_test, predicted_bayes)

0.04

In [48]:
pickle.dump(clf_neural, open("softmax.pkl", "wb"))

predicted_nn = clf_neural.predict(X_test)
result_softmax = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted_nn})

metrics.accuracy_score(y_test, predicted_nn)

0.08

In [49]:
result_softmax.head()

,true_labels,predicted_labels
62,imagerie,gerontologie
40,clinique,bloc-operatoire
94,covid-et-grossesse,gerontologie
18,biologie,bloc-operatoire
81,biologie,transfusion


## Séparateurs à Vaste Marge

Les séparateurs à vaste marge (SVM) ou encore machines à point de support sont une méthode de classification supervisée binaire. Les SVM ont été introduits par Vapnik dans son ouvrage sur la théorie de l’apprentissage statistique. Conçu comme un classifieur linéaire, la méthode consiste à chercher un hyperplan qui sépare deux classes de données de manière à maximiser la marge entres les exemples les plus proches de chacune des deux différentes classes, ces exemples sont appelés vecteurs support.

Cette technique consiste à délimiter par la frontière la plus large possible les différentes catégories des échantillons (ici les textes) de l’espace vectoriel du corpus d’apprentissage. Les vecteurs supports constituent les éléments délimitant cette frontière. Plusieurs méthodes de calcul des vecteurs support peuvent être utilisées.
* une méthode linéaire
* une méthode polynomiale
* une méthode fondée sur la loi gaussienne normale
* une méthode fondée sur la fonction sigmoïde
Nous avons effectués des essais avec plusieurs de ces méthodes.

In [50]:
kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

In [51]:
for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, data_propre.Category, test_size=0.25, random_state=42)

# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)
# Make prediction
    y_pred = svclassifier.predict(X_test)
# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))

Evaluation: Polynomial kernel
                                 precision    recall  f1-score   support

                       biologie       0.00      0.00      0.00       5.0
                bloc-operatoire       0.00      0.00      0.00       2.0
                       clinique       0.00      0.00      0.00       3.0
             covid-et-encologie       0.00      0.00      0.00       1.0
             covid-et-grossesse       0.00      0.00      0.00       2.0
covid-et-pathologies-chroniques       0.00      0.00      0.00       1.0
                     dietetique       0.00      0.00      0.00       1.0
epidemiologie-et-sante-publique       0.00      0.00      0.00       0.0
              gastroenterologie       0.00      0.00      0.00       1.0
                         greffe       0.00      0.00      0.00       1.0
                       imagerie       0.00      0.00      0.00       2.0
                    psychiatrie       0.00      0.00      0.00       1.0
                    

In [52]:
clf_svm = svm.LinearSVC()
X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, data_propre.Category, test_size=0.25, random_state=42)
clf_svm.fit(X_train_tfidf, data_propre.Category)
pickle.dump(clf_svm, open("svm.pkl", "wb"))

In [53]:
predicted_svm = clf_svm.predict(X_test)
result_svm = pd.DataFrame( {'true_labels': y_test,'predicted_labels': predicted_svm})
result_svm.head()

,true_labels,predicted_labels
62,imagerie,imagerie
40,clinique,clinique
94,covid-et-grossesse,covid-et-grossesse
18,biologie,biologie
81,biologie,biologie


In [54]:
%matplotlib inline


## Evaluation de la performance avec l'accuracy (précision du modèle)


L'évaluation d'un algorithme de Machine Learning est une partie essentielle de tout projet. Le modèle peut vous donner des résultats satisfaisants lorsqu'il est évalué à l'aide d'une métrique, par exemple exactitude du score, mais peut donner de mauvais résultats lorsqu'il est évalué par rapport à d'autres métriques telles que la fonction de perte ou toute autre métrique de ce type. Dans ce projet, nous utilisons la précision de la classification pour mesurer les performances de notre modèle.

La précision de la classification correspond au rapport entre le nombre de prédictions correctes et le nombre total d'échantillons d'entrée.

In [55]:
metrics.accuracy_score(y_test, predicted_bayes)


0.04

In [56]:
metrics.accuracy_score(y_test, predicted_nn)


0.08

In [57]:
metrics.accuracy_score(y_test, predicted_svm)


0.92

Ainsi, des trois algorithmes utilisés, l'algorithme SVM semble plus adapté à notre problème de classification avec une précision de 92%, contre 4% pour le Naive Bayes et 8% les réseaux de neurones.

# Conclusion

Lors de ce projet, nous avons surtout manqué de machines très performantes en terme d’espace mémoire notamment. Certains algorithmes en effet ont pris plusieurs heures pour donner des résultats. Nous avons passé en revue plusieurs méthodes de classification. Mais nous devons approfondir nos essais sur les différentes méthodes de classification. Pour améliorer nos résultats nous devons également effectuer des prétraitements plus poussés, car les classifieurs montrent leurs limites sur la plupart des corpus. En particulier nous fondons de nombreux espoirs sur :

* une application plus approfondie de la méthode de Dirichlet
* la combinaison de classifieur plus précise et plus adaptée que les procédures de vote simple présentées ici
* L’utilisation plus généralisée des bi-grammes,
* La combinaison de méthodes fondées sur les lemmes et sur les bi-grammes.
